In [10]:
using DataFrames, LowRankModels

In [16]:
data = readtable("missing_data.csv", separator = ',')

,age,_population_in_thousands_0_4_female_,_population_in_thousands_0_4_male_,_population_in_thousands_10_14_female_,_population_in_thousands_10_14_male_,_population_in_thousands_100_female_,_population_in_thousands_100_male_,_population_in_thousands_15_19_female_,_population_in_thousands_15_19_male_,_population_in_thousands_20_24_female_,_population_in_thousands_20_24_male_,_population_in_thousands_25_29_female_,_population_in_thousands_25_29_male_,_population_in_thousands_30_34_female_,_population_in_thousands_30_34_male_,_population_in_thousands_35_39_female_,_population_in_thousands_35_39_male_,_population_in_thousands_40_44_female_,_population_in_thousands_40_44_male_,_population_in_thousands_45_49_female_,_population_in_thousands_45_49_male_,_population_in_thousands_5_9_female_,_population_in_thousands_5_9_male_,_population_in_thousands_50_54_female_,_population_in_thousands_50_54_male_,_population_in_thousands_55_59_female_,_population_in_thousands_55_59_male_,_population_in_thousands_60_64_female_,_population_in_thousands_60_64_male_,_population_in_thousands_65_69_female_,_population_in_thousands_65_69_male_,_population_in_thousands_70_74_female_,_population_in_thousands_70_74_male_,_population_in_thousands_75_79_female_,_population_in_thousands_75_79_male_,_population_in_thousands_80_84_female_,_population_in_thousands_80_84_male_,_population_in_thousands_85_89_female_,_population_in_thousands_85_89_male_,_population_in_thousands_90_94_female_,_population_in_thousands_90_94_male_,_population_in_thousands_95_99_female_,_population_in_thousands_95_99_male_,FEMALE,MALE
1,56.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
2,41.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
3,0.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
4,46.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
5,47.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
6,50.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,1,0
7,46.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10159.0,10659.0,10454.0,10201.0,10632.0,11413.0,11013.0,11264.0,10689.0,10004.0,9217.0,8483.0,7561.0,6179.0,5278.0,4532.0,3641.0,3394.0,2442.0,2459.0,1441.0,1193.0,541.0,361.0,115.0,0,0
8,36.0,10306.0,10788.0,10346.0,10771.0,61.0,13.0,10570.0,11025.0,11094.0,11601.0,11011.0,11385.0,10863.0,10984.0,10352.0,10329.0,10308.0,10

In [12]:
obs = readtable("obs.csv", separator = ',', header = false)
head(obs)

,x1,x2
1,0,0
2,0,1
3,0,2
4,0,3
5,0,4
6,0,5


In [13]:
Ω = Tuple{Int64,Int64}[]
for i in 1:size(obs, 1)
    push!(Ω, (obs[i, 1] + 1, obs[i, 2] + 1))
end

3353066-element Array{Tuple{Int64,Int64},1}:
 (1, 1)     
 (1, 2)     
 (1, 3)     
 (1, 4)     
 (1, 5)     
 (1, 6)     
 (1, 7)     
 (1, 8)     
 (1, 9)     
 (1, 10)    
 (1, 11)    
 (1, 12)    
 (1, 13)    
 ⋮          
 (75973, 34)
 (75973, 35)
 (75973, 36)
 (75973, 37)
 (75973, 38)
 (75973, 39)
 (75973, 40)
 (75973, 41)
 (75973, 42)
 (75973, 43)
 (75973, 44)
 (75973, 45)

In [14]:
loss = HuberLoss()
nonneg = NonNegConstraint()
k = 10
glrm = GLRM(data, loss, nonneg, nonneg, k, obs=Ω);

# fit model
X,W,ch = fit!(glrm)

# predict Ahat
Ahat = X'*W

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /home/michael/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /home/michael/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:52
 [5] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /home/michael/.julia/v0.6/LowRankModels/src/fit.jl:18
 [6] fit!(::LowRankModels.GLRM) at /home/michael/.julia/v0.6/LowRankModels/src/fit.jl:9
 [7] include_string(::String, ::String) at ./loading.jl:515
 [8] include_string(::Module, ::String, ::String) at /home/michael/.julia/v0.6/Compat/src/Compat.jl:464
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/michael/.julia/v0.6/IJulia/src/execute_request.jl:154
 [10] eventloop(::ZMQ.Socket) at /home/michael/.jul

Fitting GLRM


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Float64}, ::Tuple{Int64}) at ./deprecated.jl:57
 [3] Ac_mul_B at ./linalg/matmul.jl:115 [inlined]
 [4] Ac_mul_B at ./linalg/rowvector.jl:227 [inlined]
 [5] #row_objective#58(::Array{Union{Int64, Range{Int64}},1}, ::Bool, ::Function, ::LowRankModels.GLRM, ::Int64, ::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}, ::Array{Float64,2}) at /home/michael/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:29
 [6] row_objective(::LowRankModels.GLRM, ::Int64, ::ArrayViews.ContiguousView{Float64,1,Array{Float64,2}}, ::Array{Float64,2}) at /home/michael/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:27 (repeats 2 times)
 [7] #fit!#79(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.ProxGradParams) at /home/michael/.julia/v0.6/LowRankModels/src/algorithms/proxgrad.jl:138
 [8] #fit!#76(::Array{Any,1}, ::Function, ::LowRankModels.GLRM) at /home/michael/.j

Iteration 10: objective value = 4.671860093571168e8


  likely near In[14]:7
  likely near In[14]:7
  likely near In[14]:7


Iteration 20: objective value = 1.8720806746696523e8


75973×45 Array{Float64,2}:
 26.8881   10318.0   10813.3   10374.5   …  332.986   155.227   1.08126   0.0
 26.4868   10295.6   10790.7   10351.3      332.256   154.902   1.14683   0.0
 26.4338   10306.9   10800.9   10363.2      332.631   155.075   1.11426   0.0
 28.4837   10295.8   10796.5   10355.3      332.053   154.954   1.07035   0.0
 28.0947   10314.4   10812.6   10373.6      333.318   155.233   1.14857   0.0
 26.8571   10309.9   10809.1   10369.0   …  332.823   155.159   1.07979   0.0
 27.5729   10310.6   10807.1   10368.8      332.631   155.167   1.10171   0.0
 26.8124   10311.6   10806.6   10373.3      332.358   155.169   1.06428   0.0
  3.43887   1170.9    1227.35   1177.86      37.715    17.6266  0.123878  0.0
  5.14007   1945.0    2038.94   1956.12      62.8303   29.2695  0.206708  0.0
 27.1963   10314.1   10809.5   10372.2   …  332.862   155.208   1.07946   0.0
 26.7841   10294.5   10791.5   10353.4      332.766   154.892   1.059     0.0
  5.14007   1945.0    2038.94   1956.

In [17]:
df = convert(DataFrame, Ahat)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45
1,26.888135470124467,10318.047471594668,10813.279104465393,10374.457681669703,10743.478343565901,26.01374458723191,7.4243964950389065,10567.095993612249,11033.945414802485,11139.191307429126,11624.450525779072,10980.538952607623,11429.59017180922,10865.319771397959,10986.184748085858,10361.79309633352,10327.979310097968,10288.307245094415,10158.775811013982,10688.572638205658,10495.359460660251,10210.599704319557,10635.99491004059,11444.823677695518,11021.67789199765,11293.697350063638,10698.958573475658,10018.281373834654,9192.29307266694,8450.57445878817,7534.164072921302,6154.0258017533015,5251.3240093073055,4503.700834684003,3684.7017714583567,3375.6280637427317,2479.222542487107,2494.2935317306433,1411.812352189286,1156.3405055400058,577.6406796856157,332.98611005118045,155.22732845234395,1.0812558569002797,0.0
2,26.48680796921888,10295.59952252308,10790.68654947155,10351.32850284755,10721.826733597065,25.95942501447372,7.35973282398723,10544.567490592757,11013.773983383973,11115.269489621254,11601.051309230095,10956.279279306254,11406.137001625637,10842.36056548861,10961.3730774385,10337.815490460784,10305.851881352908,10265.705517289061,10139.359418274411,10668.34000736639,10474.511103953706,10188.178275342643,10612.981523890181,11424.605584430552,10996.569210808926,11269.894552491112,10675.657858198141,9995.055026505084,9171.246312824433,8433.201031008022,7518.927927036546,6143.031490459012,5238.426843566635,4498.223015336235,3677.44862427856,3369.6799460711777,2475.119693919918,2489.6869515988524,1408.8679901613145,1153.9290071718988,576.5878855887896,332.25551343364947,154.90224581392891,1.1468281452049236,0.0
3,26.43379548286297,10306.948590254475,10800.923938173903,10363.156133037022,10733.699283833557,25.988546529412922,7.240990179843366,10556.992040073419,11025.941145747533,11130.71118295306,11615.370102059012,10966.523743607891,11419.017408258322,10854.672261554328,10973.007152250822,10352.604348965533,10320.249662305288,10277.229349383972,10148.555838882972,10676.680027079663,10484.49833900341,10199.613128166613,10626.724349998716,11434.235040406693,11010.983535833226,11283.171466447178,10688.389576351634,10007.279543427523,9184.193292976319,8441.589608840924,7526.086416322972,6147.316663204746,5245.530616150517,4500.148147735488,3683.1800322491595,3371.403608370542,2476.5326939206443,2493.8004179067852,1410.4483803918731,1155.2273441501789,573.973900104778,332.63094857224615,155.07456359936796,1.1142559476870524,0.0
4,28.483731700638884,10295.846146008349,10796.533851022166,10355.256062589346,10726.03676702962,25.96803093626658,7.21559232712117,10547.768253772087,11012.852039720823,11118.56273666904,11600.385735559283,10966.895103542916,11409.045662518736,10845.910238679808,10968.290697897564,10343.924527663441,10306.128715550458,10270.175221507741,10140.19500163283,10673.985673723531,10481.08389606475,10192.955373592704,10614.050844864862,11427.883941400029,10999.219252299583,11275.510481744297,10678.907818770276,9999.348419231223,9174.31342377513,8437.533230224442,7520.816714898068,6143.783209188312,5238.837997018956,4494.667319642483,3678.9940689039927,3371.5904811900564,2478.4251169245003,2489.4313620174007,1409.3381171051506,1154.3145027449887,579.3909253687307,332.0534287567759,154.9542663095267,1.0703489518054194,0.0
5,28.09468617922003,10314.394837738246,10812.611000693852,10373.637041020076,10745.16192732984,26.014984810002463,7.259219567170617,10566.459616444046,11039.7725598961,11138.716518394862,11625.602315741715,10984.160951804755,11431.001503173404,10862.50432454822,10985.581161176076,10362.95116100954,10326.907478890804,10288.577487428514,10158.719084301221,10693.083003690625,10499.447676180358,10210.857650056683,10634.69793508558,11448.770059221544,11018.004311657018,11294.09302800886,10697.51839669796,10016.55582122815,9191

In [19]:
writetable("glrm_results.csv",df)